# Using Beautiful Soup

In [8]:
# from urllib.request import urlopen
# from bs4 import BeautifulSoup
# import pandas as pd

# url = "http://history.basketballmonster.com/Season?seasonId=51&cats=9"
# page = urlopen(url)
# soup = BeautifulSoup(page, "html.parser")

# print(soup.prettify())

# table = soup.find("table", attrs={"class":"seasonDetailsT"})

# table.prettify()

# data = []
# rows = table.find_all('tr')
# for row in rows:
#     cols = row.find_all('td') or row.find_all('label') 
#     cols = [ele.text.strip() for ele in cols]
#     data.append([ele for ele in cols if ele])
# df = pd.DataFrame(data)
# df.head()

# Using Pandas

In [9]:
# import pandas as pd
# from urllib
# from bs4 import BeautifulSoup

# leads = [
#   ("http://history.basketballmonster.com/Season?seasonId=18&cats=9",
#    "2009-2010"),
#   ("http://history.basketballmonster.com/Season?seasonId=27&cats=9",
#    "2010-2011"),
#   ("http://history.basketballmonster.com/Season?seasonId=32&cats=9",
#    "2011-2012"),
#   ("http://history.basketballmonster.com/Season?seasonId=36&cats=9",
#    "2012-2013"),
#   ("http://history.basketballmonster.com/Season?seasonId=44&cats=9",
#    "2013-2014"),
#   ("http://history.basketballmonster.com/Season?seasonId=50&cats=9",
#    "2014-2015"),
#   ("http://history.basketballmonster.com/Season?seasonId=51&cats=9",
#    "2015-2016")
# ]

# results = pd.DataFrame()

# for lead in leads:
    
#   url = lead[0]
#   season = lead[1]

#   # Read all tables on url, use row 0 as the headers 
#   df = pd.read_html(url, header=0) 

#   # Drop table 0, unneeded
#   df = df[1]

#   # Lower-case all column names
#   df.columns = map(str.lower, df.columns)
#   df.columns = map(str.strip, df.columns)
  
#   # Drop repeated header rows, 2nd method prefered
#   # df = df.drop_duplicates(keep=False)
#   # df = df[df.player != 'player']
  
#   # Add a new column to record the season
#   df['season'] = pd.Series()
#   df['season'] = season
  
#   # Print general info
#   print(df.shape)
    
#   if results.empty:
#     results = df
#   else:
#     results = results.append(df)

# print(results.shape)
# print(results.columns)
# results.head(10)

# Extract Stats from Basketball-Reference.com

## Definitions & Setup

### Imports

In [49]:
import os
import sys
import urllib
import pickle
import pandas as pd
from bs4 import BeautifulSoup

### Save/load dictionaries

In [51]:
test = {
    'words': """
        Lorem ipsum dolor sit amet, consectetur adipiscing 
        elit. Mauris adipiscing adipiscing placerat. 
        Vestibulum augue augue, 
        pellentesque quis sollicitudin id, adipiscing.
        """,
    'list': list(range(10000)),
    'dict': dict((str(i),'a') for i in range(10000)),
    'int': 100,
    'float': 100.123456
}

def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

def get_file_size(filename):
    statinfo = os.stat(filename)
    return sizeof_fmt(statinfo.st_size)

def save_dict(dictionary, filename):
    with open(filename, 'wb') as file:
        pickle.dump(dictionary, file)
    print('\n', 'file size: ', get_file_size(filename))
    return True;

def load_dict(filename):
    print('\n', 'file size: ', get_file_size(filename))
    with open(filename, 'rb') as file:
        return pickle.load(file)

save_dict(test, 'test.pickle')

len(load_dict('test.pickle'))


 file size:  183.8 KiB

 file size:  183.8 KiB


5

### Define & Load Constants

In [75]:
# players_2015_2016 = [
#         'Acy, Quincy',
# ]

# save_dict(players_2015_2016, 'players_2015_2016.pickle')


 file size:  10.4 KiB


True

In [76]:
# save_dict(hall_of_famers, 'hall_of_famers.pickle')
# save_dict(retired_all_stars, 'retired_all_stars.pickle')
# save_dict(retired_all_nbas, 'retired_all_nbas.pickle')

# Tables to retrieve for each player, by table html ids
table_ids = [
  'per_game',
  'totals',
  'per_minute', # per 36 minutes
  'per_poss', # per 100 possessions
  'advanced', # advanced
    
  'playoffs_per_game',
  'playoffs_totals',
  'playoffs_per_minute', # playoffs per 36 minutes
  'playoffs_per_poss', # playoffs per 100 possessions
  'playoffs_advanced', 
    
  'all_star',
  'all_college_stats',
  'all_salaries',
]

# save_dict(hall_of_famers, 'hall_of_famers.pickle')
# save_dict(retired_all_stars, 'retired_all_stars.pickle')
# save_dict(retired_all_nbas, 'retired_all_nbas.pickle')

# Load list of player (names) 
hall_of_famers = load_dict('hall_of_famers.pickle')
retired_all_stars = load_dict('retired_all_stars.pickle')
retired_all_nbas = load_dict('retired_all_nbas.pickle')
players_2015_2016 = load_dict('players_2015_2016.pickle')


 file size:  3.5 KiB

 file size:  7.0 KiB

 file size:  3.6 KiB

 file size:  10.4 KiB


### Get urls from a list of player names

In [69]:
from google import search
import urllib
from bs4 import BeautifulSoup

def get_url_title(url):
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, "html.parser")
    return soup.title.text

def get_urls(player_names, outputs):
    pd.set_option('display.max_columns', None)
    
    player_names = set(player_names)
    
    for i, player_name in enumerate(player_names, 1):
        
        query = (
            'site:www.basketball-reference.com/players/*/*.html '
            '{player_name} Stats').format(player_name=player_name)
        print(i)
        print('query: ', query)
            
        while True:
            try:
                results = search(query=query, start=0, stop=1)
                urls = list(results)        
            except Exception as e:
                print('retrying...')
                continue
            break    
            
        if len(urls) < 1:
            print('url found: None')
            outputs[player_name] = None;
        else:
            print('url found: ', urls[0])
            outputs[player_name] = urls[0];
        
        print(' ')
                
        # Save outputs to temp for every 5 urls processed.
        if(i % 10 == 0):
            save_dict(outputs, 'url_outputs_temp.pickle')
    
    # Save final output
    save_dict(outputs, 'url_outputs_final.pickle')

    print('\n', 'Extracting urls from player names COMPLETED!', '\n')
    print('# of names found: ', sum((url is not None for url in outputs.values())))
    print('# of names NOT found: ', sum((url is None for url in outputs.values())))
        
player_names = hall_of_famers[0:3]
player_urls = {}

get_urls(player_names, player_urls)

1
query:  site:www.basketball-reference.com/players/*/*.html Chuck Hyatt Stats
url found: None
 
2
query:  site:www.basketball-reference.com/players/*/*.html Hank Luisetti Stats
url found: None
 
3
query:  site:www.basketball-reference.com/players/*/*.html George Mikan Stats
url found:  https://www.basketball-reference.com/players/m/mikange01.html
 

 file size:  135.0 B

 Extracting urls from player names COMPLETED! 

# of names found:  1
# of names NOT found:  2


### Get stats table from a list of (player stats) urls

In [86]:
import urllib

def get_tables(urls, outputs):
    pd.set_option('display.max_columns', None)

    for i, url in enumerate(urls, 1):
    
        page = urllib.request.urlopen(url)
        urlHtml = page.read().decode()
        
        # Get the player name
        soup = BeautifulSoup(urlHtml, "html.parser")
        player_name = soup.find("h1").text
        
        # Set the url
        outputs.setdefault(player_name, {}).setdefault('url', url);

        # Uncomment the tables
        uncommentedUrlHtml = urlHtml.replace('-->', '')
        uncommentedUrlHtml = uncommentedUrlHtml.replace('<!--', '')

        for table_id in table_ids:
            list_of_df = []
            try:
                list_of_df = pd.read_html(
                    uncommentedUrlHtml, 
                    header=0, 
                    attrs={'id': table_id})
            except ValueError as err:
                # Set failures
                outputs.setdefault(player_name, {}).setdefault('failures', {}).update({table_id: str(err)})
                continue;

            # Drop 'Unnamed' columns
            for df in list_of_df:
              df.drop([col_name for col_name in df.columns if 'Unnamed' in col_name], axis=1, inplace=True)

            # Set table
            outputs.setdefault(player_name, {}).setdefault('tables', {}).update({table_id: list_of_df[0]})
                
        # Print processing info
        print(i, '.', player_name)
        print('Tables Found: ', len(outputs[player_name].get('tables', {})), 
              ' | Failures: ', outputs[player_name].get('failures', {}).keys())
        print(' ')
        
        # Save outputs to temp for every 5 urls processed.
        if(i % 10 == 0):
            save_dict(outputs, 'data_outputs_temp.pickle')
    
    # Save final output
    save_dict(outputs, 'data_outputs_final.pickle')
    
    # Print summary info
    print('\n', 'Extracting tables from URLS COMPLETED!', '\n')
    print('# of players parsed: ', len(outputs))

player_data = {}

urls = [
    "https://www.basketball-reference.com/players/m/moncrsi01.html",
    "https://www.basketball-reference.com/players/b/bellawa01.html",
]

get_tables(urls, player_data)

1 . Sidney Moncrief
Tables Found:  13  | Failures:  dict_keys([])
 
2 . Walt Bellamy
Tables Found:  11  | Failures:  dict_keys(['playoffs_per_poss', 'all_salaries'])
 

 file size:  85.5 KiB

 Extracting tables from URLS COMPLETED! 

# of players parsed:  2


In [79]:
player_data['Walt Bellamy']['tables']['per_game']

,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,FG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,PF,PTS
0,1961-62,22.0,CHP,NBA,C,79,NaN,42.3,12.3,23.7,0.519,6.9,10.8,0.644,NaN,NaN,19.0,2.7,NaN,NaN,3.6,31.6
1,1962-63,23.0,CHZ,NBA,C,80,NaN,41.3,10.5,19.9,0.527,6.9,10.3,0.674,NaN,NaN,16.4,2.9,NaN,NaN,3.5,27.9
2,1963-64,24.0,BAL,NBA,C,80,NaN,42.4,10.1,19.8,0.513,6.7,10.3,0.651,NaN,NaN,17.0,1.6,NaN,NaN,3.8,27.0
3,1964-65,25.0,BAL,NBA,C,80,NaN,41.3,9.2,18.0,0.509,6.4,9.4,0.685,NaN,NaN,14.6,2.4,NaN,NaN,3.3,24.8
4,1965-66,26.0,TOT,NBA,C,80,NaN,41.9,8.7,17.2,0.506,5.4,8.6,0.624,NaN,NaN,15.7,2.9,NaN,NaN,3.7,22.8
5,1965-66,26.0,BAL,NBA,C,8,NaN,33.5,7.0,15.5,0.452,5.0,8.4,0.597,NaN,NaN,12.8,2.3,NaN,NaN,4.0,19.0
6,1965-66,26.0,NYK,NBA,C,72,NaN,42.8,8.9,17.3,0.512,5.4,8.6,0.627,NaN,NaN,16.0,3.0,NaN,NaN,3.6,23.2
7,1966-67,27.0,NYK,NBA,C,79,NaN,38.1,7.2,13.7,0.521,4.7,7.3,0.636,NaN,NaN,13.5,2.6,NaN,NaN,3.5,19.0
8,1967-68,28.0,NYK,NBA,C,82,NaN,32.9,6.2,11.5,0.541,4.3,6.5,0.662,NaN,NaN,11.7,2.0,NaN,NaN,3.2,16.7
9,1968-69,29.0,TOT,NBA,C,88,NaN,35.9,6.4,12.5,0.510,4.6,7.0,0.649,NaN,NaN,12.5,2.0,NaN,NaN,3.6,17.4


In [80]:
player_data['Walt Bellamy']['failures']

{'all_salaries': 'No tables found', 'playoffs_per_poss': 'No tables found'}

In [81]:
player_data['Walt Bellamy']['url']

'https://www.basketball-reference.com/players/b/bellawa01.html'

## Run Tasks

### Get Urls

In [73]:
hall_of_famers_urls = {}

get_urls(hall_of_famers, hall_of_famers_urls)

1
query:  site:www.basketball-reference.com/players/*/*.html Nikos Galis Stats
url found: None
 
2
query:  site:www.basketball-reference.com/players/*/*.html Bobby Wanzer Stats
url found:  https://www.basketball-reference.com/players/w/wanzebo01.html
 
3
query:  site:www.basketball-reference.com/players/*/*.html Arvydas Sabonis Stats
url found:  https://www.basketball-reference.com/players/s/sabonar01.html
 
4
query:  site:www.basketball-reference.com/players/*/*.html Mitch Richmond Stats
url found:  https://www.basketball-reference.com/players/r/richmmi01.html
 
5
query:  site:www.basketball-reference.com/players/*/*.html Bailey Howell Stats
url found:  https://www.basketball-reference.com/players/h/howelba01.html
 
6
query:  site:www.basketball-reference.com/players/*/*.html K.C. Jones Stats
url found:  https://www.basketball-reference.com/players/j/joneskc01.html
 
7
query:  site:www.basketball-reference.com/players/*/*.html Oscar Robertson Stats
url found:  https://www.basketball-r

url found:  https://www.basketball-reference.com/players/r/ramsefr01.html
 
57
query:  site:www.basketball-reference.com/players/*/*.html Harry Gallatin Stats
url found:  https://www.basketball-reference.com/players/g/gallaha01.html
 
58
query:  site:www.basketball-reference.com/players/*/*.html Shaquille O'Neal Stats
url found:  https://www.basketball-reference.com/players/o/onealsh01.html
 
59
query:  site:www.basketball-reference.com/players/*/*.html Scottie Pippen Stats
url found:  https://www.basketball-reference.com/players/p/pippesc01.html
 
60
query:  site:www.basketball-reference.com/players/*/*.html Robert Vandivier Stats
url found: None
 

 file size:  4.5 KiB
61
query:  site:www.basketball-reference.com/players/*/*.html Bill Sharman Stats
url found:  https://www.basketball-reference.com/players/s/sharmbi01.html
 
62
query:  site:www.basketball-reference.com/players/*/*.html Lusia Harris-Stewart Stats
url found: None
 
63
query:  site:www.basketball-reference.com/players/*/*

url found:  https://www.basketball-reference.com/players/a/arizipa01.html
 
115
query:  site:www.basketball-reference.com/players/*/*.html Zelmo Beaty Stats
url found:  https://www.basketball-reference.com/players/b/beatyze01.html
 
116
query:  site:www.basketball-reference.com/players/*/*.html Goose Tatum Stats
url found: None
 
117
query:  site:www.basketball-reference.com/players/*/*.html Robert Gruenig Stats
url found: None
 
118
query:  site:www.basketball-reference.com/players/*/*.html Hakeem Olajuwon Stats
url found:  https://www.basketball-reference.com/players/o/olajuha01.html
 
119
query:  site:www.basketball-reference.com/players/*/*.html Joe Dumars Stats
url found:  https://www.basketball-reference.com/players/d/dumarjo01.html
 
120
query:  site:www.basketball-reference.com/players/*/*.html Dawn Staley Stats
url found: None
 

 file size:  8.4 KiB
121
query:  site:www.basketball-reference.com/players/*/*.html Oscar Schmidt Stats
url found: None
 
122
query:  site:www.basket

KeyboardInterrupt: 

In [ ]:
hall_of_famers_urls.remove('https://www.basketball-reference.com/players/t/thomais02.html')
hall_of_famers_urls.append('https://www.basketball-reference.com/players/t/thomais01.html')

In [ ]:
retired_all_stars_urls = {}

extract_stats_urls(retired_all_stars, retired_all_stars_urls)

In [ ]:
retired_all_nbas_urls = {}

extract_stats_urls(retired_all_nbas, retired_all_nbas_urls)

In [77]:
players_2015_2016_urls = {}

extract_stats_urls(players_2015_2016, players_2015_2016_urls)
save_dict(players_2015_2016_urls, 'players_2015_2016_urls.pickle')

1
query:  site:www.basketball-reference.com/players/*/*.html Montero, Luis Stats
url found:  https://www.basketball-reference.com/players/m/montelu01.html
 
2
query:  site:www.basketball-reference.com/players/*/*.html Johnson, James Stats
url found:  https://www.basketball-reference.com/players/j/johnsja01.html
 
3
query:  site:www.basketball-reference.com/players/*/*.html Thompson, Jason Stats
url found:  https://www.basketball-reference.com/players/t/thompja02.html
 
4
query:  site:www.basketball-reference.com/players/*/*.html Hilliard, Darrun Stats
url found:  https://www.basketball-reference.com/players/h/hillida01.html
 
5
query:  site:www.basketball-reference.com/players/*/*.html Wright, Delon Stats
url found:  https://www.basketball-reference.com/players/w/wrighde01.html
 
6
query:  site:www.basketball-reference.com/players/*/*.html Cauley-Stein, Willie Stats
url found:  https://www.basketball-reference.com/players/c/caulewi01.html
 
7
query:  site:www.basketball-reference.com/p

url found:  https://www.basketball-reference.com/players/h/hinriki01.html
 
53
query:  site:www.basketball-reference.com/players/*/*.html Korver, Kyle Stats
url found:  https://www.basketball-reference.com/players/k/korveky01.html
 
54
query:  site:www.basketball-reference.com/players/*/*.html Cunningham, Dante Stats
url found:  https://www.basketball-reference.com/players/c/cunnida01.html
 
55
query:  site:www.basketball-reference.com/players/*/*.html Jackson, Reggie Stats
url found:  https://www.basketball-reference.com/players/j/jacksre01.html
 
56
query:  site:www.basketball-reference.com/players/*/*.html Pressey, Phil Stats
url found:  https://www.basketball-reference.com/players/p/pressph01.html
 
57
query:  site:www.basketball-reference.com/players/*/*.html Dejean-Jones, Bryce Stats
url found:  https://www.basketball-reference.com/players/d/dejeabr01.html
 
58
query:  site:www.basketball-reference.com/players/*/*.html Galloway, Langston Stats
url found:  https://www.basketball-r

url found:  https://www.basketball-reference.com/players/m/millemi01.html
 
104
query:  site:www.basketball-reference.com/players/*/*.html Munford, Xavier Stats
url found:  https://www.basketball-reference.com/players/m/munfoxa02.html
 
105
query:  site:www.basketball-reference.com/players/*/*.html Roberson, Andre Stats
url found:  https://www.basketball-reference.com/players/r/roberan03.html
 
106
query:  site:www.basketball-reference.com/players/*/*.html Bryant, Kobe Stats
url found:  https://www.basketball-reference.com/players/b/bryanko01.html
 
107
query:  site:www.basketball-reference.com/players/*/*.html Iguodala, Andre Stats
url found:  https://www.basketball-reference.com/players/i/iguodan01.html
 
108
query:  site:www.basketball-reference.com/players/*/*.html Harris, Gary Stats
url found:  https://www.basketball-reference.com/players/h/harriga01.html
 
109
query:  site:www.basketball-reference.com/players/*/*.html McRoberts, Josh Stats
url found:  https://www.basketball-refer

url found:  https://www.basketball-reference.com/players/w/warretj01.html
 
155
query:  site:www.basketball-reference.com/players/*/*.html Turner, Myles Stats
url found:  https://www.basketball-reference.com/players/t/turnemy01.html
 
156
query:  site:www.basketball-reference.com/players/*/*.html Faried, Kenneth Stats
url found:  https://www.basketball-reference.com/players/f/farieke01.html
 
157
query:  site:www.basketball-reference.com/players/*/*.html Irving, Kyrie Stats
url found:  https://www.basketball-reference.com/players/i/irvinky01.html
 
158
query:  site:www.basketball-reference.com/players/*/*.html Olynyk, Kelly Stats
url found:  https://www.basketball-reference.com/players/o/olynyke01.html
 
159
query:  site:www.basketball-reference.com/players/*/*.html Gasol, Marc Stats
url found:  https://www.basketball-reference.com/players/g/gasolma01.html
 
160
query:  site:www.basketball-reference.com/players/*/*.html Smith, Jason Stats
url found:  https://www.basketball-reference.co

url found:  https://www.basketball-reference.com/players/g/greenja01.html
 
206
query:  site:www.basketball-reference.com/players/*/*.html Crawford, Jamal Stats
url found:  https://www.basketball-reference.com/players/c/crawfja01.html
 
207
query:  site:www.basketball-reference.com/players/*/*.html Gooden, Drew Stats
url found:  https://www.basketball-reference.com/players/g/goodedr01.html
 
208
query:  site:www.basketball-reference.com/players/*/*.html Belinelli, Marco Stats
url found:  https://www.basketball-reference.com/players/b/belinma01.html
 
209
query:  site:www.basketball-reference.com/players/*/*.html Butler, Rasual Stats
url found:  https://www.basketball-reference.com/players/b/butlera01.html
 
210
query:  site:www.basketball-reference.com/players/*/*.html Wroten, Tony Stats
url found:  https://www.basketball-reference.com/players/w/wroteto01.html
 

 file size:  18.7 KiB
211
query:  site:www.basketball-reference.com/players/*/*.html Thompson, Klay Stats
url found:  https:

...

url found:  https://www.basketball-reference.com/players/l/lopezbr01.html
 
308
query:  site:www.basketball-reference.com/players/*/*.html Jack, Jarrett Stats
url found:  https://www.basketball-reference.com/players/j/jackja01.html
 
309
query:  site:www.basketball-reference.com/players/*/*.html Amundson, Louis Stats
url found:  https://www.basketball-reference.com/players/a/amundlo01.html
 
310
query:  site:www.basketball-reference.com/players/*/*.html Ezeli, Festus Stats
url found:  https://www.basketball-reference.com/players/e/ezelife01.html
 

 file size:  27.9 KiB
311
query:  site:www.basketball-reference.com/players/*/*.html Kidd-Gilchrist, Michael Stats
url found:  https://www.basketball-reference.com/players/k/kiddgmi01.html
 
312
query:  site:www.basketball-reference.com/players/*/*.html Bogdanovic, Bojan Stats
url found:  https://www.basketball-reference.com/players/b/bogdabo02.html
 
313
query:  site:www.basketball-reference.com/players/*/*.html Mbah a Moute, Luc Stats
url 

url found:  https://www.basketball-reference.com/players/c/casspom01.html
 
359
query:  site:www.basketball-reference.com/players/*/*.html George, Paul Stats
url found:  https://www.basketball-reference.com/players/g/georgpa01.html
 
360
query:  site:www.basketball-reference.com/players/*/*.html Knight, Brandon Stats
url found:  https://www.basketball-reference.com/players/k/knighbr03.html
 

 file size:  32.6 KiB
361
query:  site:www.basketball-reference.com/players/*/*.html Aminu, Al-Farouq Stats
url found:  https://www.basketball-reference.com/players/a/aminual01.html
 
362
query:  site:www.basketball-reference.com/players/*/*.html Acy, Quincy Stats
url found:  https://www.basketball-reference.com/players/a/acyqu01.html
 
363
query:  site:www.basketball-reference.com/players/*/*.html Connaughton, Pat Stats
url found:  https://www.basketball-reference.com/players/c/connapa01.html
 
364
query:  site:www.basketball-reference.com/players/*/*.html Zeller, Tyler Stats
url found:  https://

url found:  https://www.basketball-reference.com/players/h/hillso01.html
 
410
query:  site:www.basketball-reference.com/players/*/*.html Love, Kevin Stats
url found:  https://www.basketball-reference.com/players/l/loveke01.html
 

 file size:  37.2 KiB
411
query:  site:www.basketball-reference.com/players/*/*.html Gasol, Pau Stats
url found:  https://www.basketball-reference.com/players/g/gasolpa01.html
 
412
query:  site:www.basketball-reference.com/players/*/*.html Leonard, Meyers Stats
url found:  https://www.basketball-reference.com/players/l/leoname01.html
 
413
query:  site:www.basketball-reference.com/players/*/*.html Dinwiddie, Spencer Stats
url found:  https://www.basketball-reference.com/players/d/dinwisp01.html
 
414
query:  site:www.basketball-reference.com/players/*/*.html Collison, Nick Stats
url found:  https://www.basketball-reference.com/players/c/collini01.html
 
415
query:  site:www.basketball-reference.com/players/*/*.html Valanciunas, Jonas Stats
url found:  https

url found:  https://www.basketball-reference.com/players/s/smithgr02.html
 

 file size:  41.8 KiB
461
query:  site:www.basketball-reference.com/players/*/*.html Grant, Jerami Stats
url found:  https://www.basketball-reference.com/players/g/grantje01.html
 
462
query:  site:www.basketball-reference.com/players/*/*.html Gibson, Taj Stats
url found:  https://www.basketball-reference.com/players/g/gibsota01.html
 
463
query:  site:www.basketball-reference.com/players/*/*.html Caldwell-Pope, Kentavious Stats
url found:  https://www.basketball-reference.com/players/c/caldwke01.html
 
464
query:  site:www.basketball-reference.com/players/*/*.html Young, James Stats
url found:  https://www.basketball-reference.com/players/y/youngja01.html
 
465
query:  site:www.basketball-reference.com/players/*/*.html Young, Nick Stats
url found:  https://www.basketball-reference.com/players/y/youngni01.html
 
466
query:  site:www.basketball-reference.com/players/*/*.html Gay, Rudy Stats
url found:  https://

True

### Get Data

In [ ]:
hall_of_famers_data = {}

get_tables(hall_of_famers_urls.values(), hall_of_famers_data)
save_dict(hall_of_famers_data, 'hall_of_famers_data.pickle')

In [83]:
players_2015_2016_data = {}

get_tables(players_2015_2016_urls.values(), players_2015_2016_data)
save_dict(players_2015_2016_data, 'players_2015_2016_data.pickle')

1 . Luis Montero
Tables Found:  11  | Failures:  2
 
2 . James Johnson
Tables Found:  12  | Failures:  1
 
3 . Jason Thompson
Tables Found:  12  | Failures:  1
 
4 . Darrun Hilliard
Tables Found:  7  | Failures:  6
 
5 . Delon Wright
Tables Found:  12  | Failures:  1
 
6 . Willie Cauley-Stein
Tables Found:  7  | Failures:  6
 
7 . Robert Covington
Tables Found:  7  | Failures:  6
 


KeyboardInterrupt: 

### Get Stats for All

In [ ]:
import pickle

retired_stars_urls = []

with open('retired_stars_urls.pickle', 'rb') as file:
    retired_stars_urls = pickle.load(file)
    
len(retired_stars_urls)

In [ ]:
# retired_stars_urls = list(set(hall_of_famers_urls).union(set(retired_all_stars_urls)).union(set(retired_all_nbas_urls)))
retired_stars_stats = {}
retired_stars_failures = {}

parse_urls_into_tables(retired_stars_urls, retired_stars_stats, retired_stars_failures)

In [ ]:
len(retired_stars_stats)

In [ ]:
# retired_stars_stats['Walt Bellamy']['all_salaries']

retired_stars_stats['Zydrunas Ilgauskas']['per_game']

In [ ]:
save_dict(retired_stars_stats, 'retired_stars_stats.pickle')

In [ ]:
save_dict(retired_stars_urls, 'retired_stars_urls.pickle')

In [ ]:
retired_stars_urls = load_dict('retired_stars_urls.pickle')
len(retired_stars_urls)